In [0]:
%tensorflow_version 1.x
from keras.models import load_model
from __future__ import print_function
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import applications
import cv2 
from os import listdir
import numpy as np
import matplotlib. pyplot as plt
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D,GlobalAveragePooling2D,Activation,BatchNormalization,Dropout,Concatenate
from keras.models import Model
from keras.applications import DenseNet121
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import Adam, Adamax
from keras.callbacks import LearningRateScheduler
from keras import optimizers
#Augmentation
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, EarlyStopping
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import train_test_split
import urllib,math

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def augment(images,num):
  aug = []
  datagens = []
  datagens.append(ImageDataGenerator(width_shift_range=0.15))
  datagens.append(ImageDataGenerator(height_shift_range=0.15))
  datagens.append(ImageDataGenerator(horizontal_flip=True))
  datagens.append(ImageDataGenerator(rotation_range=45))
  datagens.append(ImageDataGenerator(brightness_range=[0.6,1]))
  datagens.append(ImageDataGenerator(zoom_range=0.15))
  for data in images:
    samples = expand_dims(data, 0)
    for datagen in datagens:
      it = datagen.flow(samples, batch_size=1)
      for i in range(num):
        batch = it.next()
        image = batch[0].astype('uint8')
        aug.append(image)
  return aug

In [0]:
x_train=np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/x_train.npy')
y_train1=np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/y_train.npy')
x_test=np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/x_test.npy')
y_test1=np.load('/content/drive/My Drive/Corona_Image/Pavel/Data/v3/y_test.npy')

In [0]:
print(np.unique(y_train1,return_counts=True))
print(np.unique(y_test1,return_counts=True))

(array([0, 1, 2], dtype=int32), array([7966, 5451,  207]))
(array([0, 1, 2], dtype=int32), array([885, 594,  31]))


In [0]:
def make_two_class(arr):
  arr[arr == 1] = 0
  arr[arr == 2] = 1
  return arr

In [0]:
y_train=make_two_class(y_train1)
y_test=make_two_class(y_test1)
print(np.unique(y_train1,return_counts=True))
print(np.unique(y_test1,return_counts=True))

(array([0, 1], dtype=int32), array([13417,   207]))
(array([0, 1], dtype=int32), array([1479,   31]))


In [0]:
y_train_cat = np_utils.to_categorical(y_train, 2)
train_data, val_data, train_label, val_label = train_test_split(x_train, y_train_cat, test_size=0.1, random_state=42, stratify=y_train_cat)
train_label = np.argmax(train_label,axis=1)
corona = train_data[train_label==1]
aug = np.array(augment(corona,9))
label = np.empty(aug.shape[0],dtype='int32')
label.fill(1)
train_data = np.concatenate((train_data, aug), axis=0)
train_label = np.concatenate((train_label, label), axis=0)
train_data,train_label=shuffle(train_data,train_label)

In [0]:
print(np.unique(train_label,return_counts=True))
train_label = np_utils.to_categorical(train_label, 2)
print(train_data.shape,train_label.shape)

(array([0, 1]), array([12075, 10230]))
(22305, 224, 224, 3) (22305, 2)


In [0]:
val_label = np.argmax(val_label,axis=1)
corona = val_data[val_label==1]
aug = np.array(augment(corona,9))
label = np.empty(aug.shape[0],dtype='int32')
label.fill(1)
val_data = np.concatenate((val_data, aug), axis=0)
val_label = np.concatenate((val_label, label), axis=0)
print(np.unique(val_label,return_counts=True))
val_label = np_utils.to_categorical(val_label, 2)
val_data,val_label=shuffle(val_data,val_label)
print(val_data.shape,val_label.shape)

(array([0, 1]), array([1342, 1155]))
(2497, 224, 224, 3) (2497, 2)


In [0]:
densenet = DenseNet121(weights='/content/drive/My Drive/Corona_Image/chexnet_weights.h5',
        # weights = None ,
        include_top=True,
        input_shape=(224,224,3),classes=14
      )
      
m = densenet.layers[-2].output
predictions = Dense(2, activation='softmax')(m)
model = Model(inputs=densenet.inputs, outputs=predictions)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.




In [0]:
from keras import backend as K

def binary_accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)))


def categorical_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=-1),
                          K.argmax(y_pred, axis=-1)))


def sparse_categorical_accuracy(y_true, y_pred):
    return K.mean(K.equal(K.max(y_true, axis=-1),
                          K.cast(K.argmax(y_pred, axis=-1), K.floatx())))


def top_k_categorical_accuracy(y_true, y_pred, k=5):
    return K.mean(K.in_top_k(y_pred, K.argmax(y_true, axis=-1), k))


def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def mean_absolute_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


def mean_absolute_percentage_error(y_true, y_pred):
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true),
                                            K.epsilon(),
                                            None))
    return 100. * K.mean(diff)


def mean_squared_logarithmic_error(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    return K.mean(K.square(first_log - second_log))


def hinge(y_true, y_pred):
    return K.mean(K.maximum(1. - y_true * y_pred, 0.))


def squared_hinge(y_true, y_pred):
    return K.mean(K.square(K.maximum(1. - y_true * y_pred, 0.)))


def categorical_crossentropy(y_true, y_pred):
    return K.mean(K.categorical_crossentropy(y_pred, y_true))


def sparse_categorical_crossentropy(y_true, y_pred):
    return K.mean(K.sparse_categorical_crossentropy(y_pred, y_true))


def binary_crossentropy(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_pred, y_true))


def kullback_leibler_divergence(y_true, y_pred):
    y_true = K.clip(y_true, K.epsilon(), 1)
    y_pred = K.clip(y_pred, K.epsilon(), 1)
    return K.mean(K.sum(y_true * K.log(y_true / y_pred), axis=-1))


def poisson(y_true, y_pred):
    return K.mean(y_pred - y_true * K.log(y_pred + K.epsilon()))


def cosine_proximity(y_true, y_pred):
    y_true = K.l2_normalize(y_true, axis=-1)
    y_pred = K.l2_normalize(y_pred, axis=-1)
    return -K.mean(y_true * y_pred)


def matthews_correlation(y_true, y_pred):
    """Matthews correlation metric.
    It is only computed as a batch-wise average, not globally.
    Computes the Matthews correlation coefficient measure for quality
    of binary classification problems.
    """
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())


def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score


def fmeasure(y_true, y_pred):
    """Computes the f-measure, the harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    """
    return fbeta_score(y_true, y_pred, beta=1)

In [0]:
adam = Adamax(learning_rate=0.00001, beta_1=0.9, beta_2=0.999)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy', precision, recall, fbeta_score])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
#class_weights = {0: 3, 1: 7}
history = model.fit(train_data, train_label, batch_size=16,
                         epochs=100,verbose=1, validation_data=(val_data,val_label), shuffle=True, class_weight=class_weights,
                           callbacks=[reduce_lr, es])


Train on 22305 samples, validate on 2497 samples
Epoch 1/100
22305/22305 [==============================] - 741s 33ms/step - loss: 0.6395 - accuracy: 0.6877 - precision: 0.6879 - recall: 0.6879 - fbeta_score: 0.6879 - val_loss: 0.5941 - val_accuracy: 0.7869 - val_precision: 0.7882 - val_recall: 0.7882 - val_fbeta_score: 0.7882
Epoch 2/100
22305/22305 [==============================] - 707s 32ms/step - loss: 0.5763 - accuracy: 0.7852 - precision: 0.7847 - recall: 0.7847 - fbeta_score: 0.7847 - val_loss: 0.5265 - val_accuracy: 0.8150 - val_precision: 0.8161 - val_recall: 0.8161 - val_fbeta_score: 0.8161
Epoch 3/100
22305/22305 [==============================] - 709s 32ms/step - loss: 0.5123 - accuracy: 0.8214 - precision: 0.8209 - recall: 0.8209 - fbeta_score: 0.8209 - val_loss: 0.4584 - val_accuracy: 0.8390 - val_precision: 0.8400 - val_recall: 0.8400 - val_fbeta_score: 0.8400
Epoch 4/100
22305/22305 [==============================] - 709s 32ms/step - loss: 0.4391 - accuracy: 0.8564 - 